COMPUTE BOM FROM SX AND NVL

In [ ]:
import pandas as pd
cols = ['Material', 'Movement type', 'Qty in unit of entry', 'Unit of Entry', 'Order']
df = pd.concat(pd.read_excel(
    '../data/TONG.XLSX', sheet_name=None, usecols=cols ,converters={
            'Material': str,
            'Order': str
        }
    ), ignore_index=True
)

df = df.loc[
    df['Movement type'].isin([101, 102, 261, 262]) &
    ~pd.isna(df['Order'])
]
sx = df.loc[df['Movement type'].isin([101, 102])]
nvl = df.loc[df['Movement type'].isin([261, 262])]
sx = sx.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
sx.reset_index(inplace=True)
sx.rename(columns={
    'Material': 'Product',
    'Unit of Entry': 'Product Unit',
    'Qty in unit of entry': 'Product Quantity'
}, inplace=True)
sx = sx.loc[sx['Product Quantity'] != 0]

nvl = nvl.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
nvl.reset_index(inplace=True)
nvl.rename(columns={
    'Unit of Entry': 'Material Unit',
    'Qty in unit of entry': 'Material Quantity'
}, inplace=True)

nvl['Material Quantity'] = -nvl['Material Quantity']
sx.drop_duplicates(inplace=True)
result = pd.merge(sx, nvl, how="left", on='Order', sort=False,
                          indicator=False, validate=None)
result = result.groupby(['Product', 'Product Unit', 'Material', 'Material Unit']).agg(
            {'Product Quantity': 'sum', 'Material Quantity': 'sum'})
result.reset_index(inplace=True)
result['BOM'] = result['Material Quantity'] / result['Product Quantity']


CONCAT IN SHEETS

In [3]:
import pandas as pd
# df = pd.concat(pd.read_excel(
#     '../data/2022-03-24_BOM_KG3.xlsx', sheet_name=None,
#     ), ignore_index=True
# )
# df = df[df['Usage'].notna()]
# df.to_excel('../output/KG3.xlsx')


# df = pd.read_excel('../data/BOM winnercom.xlsx', header=6, usecols=['Mã mẹ', 'Mã con', 'Actual BOM'], converters={'Actual BOM': float})

# result = df.groupby(['Mã mẹ', 'Mã con'], sort=False).agg(
#             {'Actual BOM': 'sum'})
# result.to_excel('../output/winnercom.xlsx')



df = pd.read_excel('../data/Bảng tính giá thành T2.xlsb', header=5)
# cols = df.columns[18:]
# df = df[cols]
df.set_index('Mã', inplace=True)
trans = df.transpose()
cols = trans.columns
trans.reset_index(inplace=True)
# trans = trans.iloc[2:]
cols = [x for x in trans.columns if not pd.isna(x)]
trans = trans[cols]

trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')
result= trans.set_index(["index"]).stack()
result = result.reset_index()
df.reset_index(inplace=True)
# df = df[['Material Name', 'Unit.1']]
# end = pd.merge(
#     result,
#     df,
#     how="left",
#     on='Material Name',
#     sort=False,
#     indicator=False,
#     validate=None,
# )
# end = end.drop(end[end[0] == 0].index)


In [8]:
df = df.loc['Mã']

,Mã,SHOES - PFC,SHOES - PBS,SHOES/PMO,URT,R,R＃,MD,MD＃,MU＃,...,Unnamed: 356,Unnamed: 357,Unnamed: 358,Unnamed: 359,Unnamed: 360,Unnamed: 361,Unnamed: 362,Unnamed: 363,Unnamed: 364,Unnamed: 365
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2K1453PBKWT02-SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2K1453PIDWT02_SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2K1802PBKWT01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2K1802PBWT01-SO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


CONCAT IN FOLDERS

In [14]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)

path = '../data/MB51'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)

cols = ['Material', 'Movement type', 'Qty in unit of entry', 'Unit of Entry', 'Order']

dfs = [pd.read_excel(x, usecols=cols) for x in file_names]
df = pd.concat(dfs)

sx = df.loc[df['Movement type'].isin([101, 102])]
nvl = df.loc[df['Movement type'].isin([261, 262])]

# CLEAN SX DFS
sx = sx.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
sx.reset_index(inplace=True)
sx.rename(columns={
    'Material': 'Product',
    'Unit of Entry': 'Product Unit',
    'Qty in unit of entry': 'Product Quantity'
}, inplace=True)
sx = sx.loc[sx['Product Quantity'] != 0]
sx.drop_duplicates(inplace=True)

# CLEAN NVL DFS
nvl = nvl.groupby(
    ['Material', 'Unit of Entry', 'Order']).agg(
        {'Qty in unit of entry': 'sum'}
)
nvl.reset_index(inplace=True)
nvl.rename(columns={
    'Unit of Entry': 'Material Unit',
    'Qty in unit of entry': 'Material Quantity'
}, inplace=True)
nvl['Material Quantity'] = -nvl['Material Quantity']


result = pd.merge(sx, nvl, how="left", on='Order', sort=False,
                          indicator=False, validate=None)
result = result.groupby(['Product', 'Product Unit', 'Material', 'Material Unit']).agg(
            {'Product Quantity': 'sum', 'Material Quantity': 'sum'})
result.reset_index(inplace=True)
result['BOM'] = result['Material Quantity'] / result['Product Quantity']
result.to_excel('../output/MB51-1.xlsx')
